In [540]:
import os
import re
import numpy as np
import pandas as pd
import plotly.express as px

In [541]:
DEST_COORDINATES = [0, 0, 1]

SAVE_PATH = "../results_perf_i1.html"

In [542]:
models = {
    "err": {"path" : "../results-nav-dist-err-rew-wi-v-5/plt/i1/", "color": "red"},
    "no-err": {"path" : "../results-nav-dist-v-3/plt/i1/", "color": "blue"},
}

In [543]:
def plot(dir, dist):
    fig = px.line_3d()
    fig.update_layout(
        title_text=f"Trajectories {dir=} {dist=}",
        scene=dict(
            # xaxis=dict(range=[-2, 2]),
            # yaxis=dict(range=[-2, 2]),
            zaxis=dict(range=[0, 2]),
        ),
    )

    ret = {
        "dir": dir,
        "dist": dist,
        "distance_travelled": {},
        "convereged": {},
    }
    for mdl in models:
        df = pd.read_csv(os.path.join(models[mdl]["path"], f"{dir}_{dist}.csv"))

        dt = 0
        for i in range(len(df) - 1):
            dt += np.linalg.norm(df.iloc[i][["x", "y", "z"]] - df.iloc[i + 1][["x", "y", "z"]])

        fig.add_scatter3d(
            x=df.x,
            y=df.y,
            z=df.z,
            line=dict(color=models[mdl]["color"]),
            marker=dict(size=3),
            name=mdl,
        )

        ret["distance_travelled"][mdl] = dt
        ret["convereged"][mdl] = (
            "Yes" if np.linalg.norm(df.iloc[-1][["x", "y", "z"]] - DEST_COORDINATES) < 0.20 else "No"
        )

        # ret[f"distance_travelled.{mdl}"] = dt
        # ret[f"convereged.{mdl}"] = "Yes" if np.linalg.norm(df.iloc[-1, :] - [0, 0, 1]) < 0.1 else "No"

    fig.show()

    return ret

In [544]:
files = os.listdir(models[list(models.keys())[0]]["path"])
files = list(filter(lambda name: name.startswith("x") or name.startswith("y") or name.startswith("z") or name.startswith("xyz"), files))

results = []

for file in files:
    name = file.split(".csv")[0]
    (dir, dist) = name.split("_")
    ret = plot(dir, dist)
    results.append(ret)

In [545]:
# cols = pd.MultiIndex.from_tuples([
#     ("dir", ""),
#     ("dist", ""),
#     ("distance_travelled", "err"),
#     ("distance_travelled", "no-err"),
#     ("converged", "err"),
#     ("converged", "no-err"),
# ])

results_df = pd.json_normalize(results)

results_df.sort_values(by=["dir", "dist"], inplace=True)

results_df

# pd.DataFrame(results_df.values, columns=cols)

results_df.to_html(SAVE_PATH)